In [1]:
import timeit
start = timeit.default_timer()

import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

randomlist = []
df_demand = {}
random.seed(1) # Use the same random seed
for i in range(0,20):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(20):
    df_demand['D'+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v
    
df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0
    
np.random.seed(1) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,20, size=(sum,20))).astype(float)

random.seed(1) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)
    
age = {}
for i in range(len(age_b)):
    age[i] = age_b[i] 
    
kpl = []
co2 = []
random.seed(1) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)
        
fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])
    
random.seed(1) # Use the same random seed
doc = {}
for i in range(0,20):
    n = random.uniform(50,100)
    doc[i] = n
    
df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]

In [2]:
#df_tdkom

In [3]:
df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]
dk = df_tdkom.transpose()
costs_t = dk.to_dict()
#costs_t

costs1=copy.deepcopy(costs_t)
cols = sorted(df_demand.keys())

In [4]:
#df_demand

In [5]:
res = dict((k, defaultdict(int)) for k in costs_t)
g = {}
for x in df_demand:
    g[x] = sorted(costs_t.keys(), key=lambda g: costs_t[g][x])

for x in df_demand:
    res[g[x][0]][x] = df_demand[x]

In [6]:
## Deleting Rows if Supply is Met after First Allocation!

#from operator import*
for x in res:
    if list(res[x].values()) == []:
        #print('Null is:',x)
        continue
    else:
        #print(x)
        result = 0
        for i in list(res[x].values()):
            result += i
            #result = add(i, result)
        #print(result)
        if result == df_supply[x]:
            del df_supply[x]
            del costs_t[x]
    #print(x)

In [7]:
while(df_supply):
    g = {}
    for x in df_demand:
        g[x] = sorted(costs_t.keys(), key=lambda g: costs_t[g][x])

    #### Step 3: To check the ER
    #from operator import*
    ER = []
    for x in df_supply:
        #### When only one cell is allocated a value in an ER
        if len(res[x].values()) == 1:
            if list(res[x].values())[0] > df_supply[x]:
                ER.append(x)
            #print(ER)
        else:
            #### When multiple cells are allocated values in ER 
            result = 0
            for i in list(res[x].values()):
                #result = add(i, result)
                result += i
            if result > df_supply[x]:
                ER.append(x)
 

    ## Step 4: To Calculate Differences for every cell in ER
    Diff = {}
    for x in ER:
        for y in res[x]:
            if len(g[y]) !=1:
                if res[x][y] !=0:
                    m = 1
                    while g[y][m] in ER:
                        m += 1
                #print(costs_t[g[y][m]][y], costs_t[g[y][0]][y])
                Diff.setdefault(x, {}).setdefault(y, costs_t[g[y][m]][y] - costs_t[g[y][0]][y])         
                
            elif len(g[y]) ==1:
                Diff.setdefault(x, {}).setdefault(y, costs_t[g[y][0]][y])
    
    #print(Diff)

    SD = {key: min(val.values()) for key, val in Diff.items()}

    SDiff = min(SD.values())

    ### Counting occurrences of SDiff
    count = 0
    for key, value in SD.items():
        if value == SDiff:
            count += 1

    ### Step 5: For two/more SDiff with same value, choosing cell with largest allocation unit!
    if count > 1:
        All = {}
        for k,v in Diff.items():
            #print(v)
            for (p,o) in v.items():
                if o == SDiff:
                    All[k] = p

        max_val = 0
        for (key,value) in All.items():
            if res[key][value] > max_val:
                max_val = res[key][value]
                new_key = key
                sub_key = value
                #print(new_key,sub_key, res[new_key][sub_key])
            else:
                continue

        new_value = SDiff
        #print(new_value, new_key, sub_key)

    if count == 1:
        new_value, new_key = min((value, key) for key, value in SD.items())

        key_list = list(Diff[new_key].keys())
        val_list = list(Diff[new_key].values())

        position = val_list.index(new_value)
        sub_key = key_list[position]

        #print(new_value, new_key, sub_key)

    FLR = new_key
    #FLR

    m = 1
    while g[sub_key][m] in ER:
        m += 1
    #print(m)
    SLR = g[sub_key][m]
    #SLR

    TCFLR = 0
    for i in list(costs_t[FLR].values()):
        TCFLR += i

    TCSLR = 0
    for i in list(costs_t[SLR].values()):
        TCSLR += i

    ### Step 9:
    if TCFLR > TCSLR:
        ## Satisfy FLR & keep the rest in SLC
        if res[FLR][sub_key] == 1:
            res[SLR][sub_key] = res[FLR][sub_key]
            del res[FLR][sub_key]
        else:
            res[SLR][sub_key] = res[FLR][sub_key] - df_supply[FLR]
            res[FLR][sub_key] = df_supply[FLR]
 

    ### Step 9:
    if TCFLR < TCSLR:
        ## Satisfy SLR & Keep the rest in FLC
        if res[FLR][sub_key] == 1:
            res[SLR][sub_key] = res[FLR][sub_key]
            del res[FLR][sub_key]
        else:
            res[FLR][sub_key] = res[FLR][sub_key] - df_supply[FLR]
            res[SLR][sub_key] = df_supply[SLR]


    result_1 = 0
    for i in list(res[FLR].values()):
        result_1 += i
    #print(result_1)

    result_2 = 0
    for i in list(res[SLR].values()):
        result_2 += i
    #print(result_2)

    if result_1 == df_supply[FLR]:
        del df_supply[FLR]
        del costs_t[FLR]

    if result_2 == df_supply[SLR]:
        del df_supply[SLR]
        del costs_t[SLR]

In [8]:
df_supply

{}

In [9]:
cost = 0
for g in sorted(costs1):
    # print (g, " ",)
    for n in cols:
        y = res[g][n]
        if y != 0:
            pass
            # print (y,)
        cost += y * costs1[g][n]
        # print ("  ",)
    # print(" ")
print ("Total Cost = ", cost)

Total Cost =  254958.26194013908


stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time))